In [44]:
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')


[nltk_data] Downloading package punkt to /home/eljan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [87]:
dataset_paths = [
    "eljanmahammadli/book-chunks-512",
    "eljanmahammadli/clean-chunks-512",
    "vrashad/various_topics_articles_azerbaijan",
    "vrashad/news_azerbaijan",
    "azcorpus/azcorpus_v0",
    "eljanmahammadli/az-corpus",
    "mHossain/azerbaijani_sum_v1",
]

In [3]:
ds0 = load_dataset(dataset_paths[0])['train']
ds0 = ds0.map(lambda example: {**example, 'source': 'book_chunks_512'})

In [4]:
ds1 = load_dataset(dataset_paths[1])['train']
ds1 = ds1.map(lambda example: {**example, 'source': 'clean-chunks-512'})

In [17]:
ds2 = load_dataset(dataset_paths[2])['train']
ds2 = ds2.rename_column("article", "text")
ds2 = ds2.map(lambda example: {**example, 'source': 'various_topics_articles'})

In [45]:
ds3 = load_dataset(dataset_paths[3])['train']
ds3 = ds3.remove_columns(["date", "id", "title"])
ds3 = ds3.map(lambda example: {**example, 'source': 'news_azerbaijan'})

In [65]:
ds4 = load_dataset(dataset_paths[4])['train']
ds4 = ds4.filter(lambda example: example['source'] == 'az_books')
ds4 = ds4.remove_columns(['reliability', 'source', 'license', 'id'])
ds4 = ds4.map(lambda example: {**example, 'source': 'azcorpus_books'})

In [89]:
ds5 = load_dataset(dataset_paths[5])['train']
ds5 = ds5.remove_columns(['url'])

In [95]:
ds6_train = load_dataset(dataset_paths[6])['train']
ds6_test = load_dataset(dataset_paths[6])['test']
ds6_train = ds6_train.remove_columns(["summary_Azerbaijani"])
ds6_test= ds6_test.remove_columns(["summary_Azerbaijani"])
ds6_train = ds6_train.rename_column("text_Azerbaijani", "text")
ds6_test = ds6_test.rename_column("text_Azerbaijani", "text")
ds6_train = ds6_train.map(lambda example: {**example, 'source': 'az_cnn'})
ds6_test = ds6_test.map(lambda example: {**example, 'source': 'az_cnn'})

Map: 100%|██████████| 1000/1000 [00:00<00:00, 15957.69 examples/s]


In [96]:
ds6_train

Dataset({
    features: ['text', 'source'],
    num_rows: 5000
})

In [106]:
dataset = concatenate_datasets([
    ds0,
    ds1,
    ds2,
    ds3,
    ds4,
    ds5,
    ds6_train,
    ds6_test
])
dataset_dict = DatasetDict({
    "train": dataset
})
dataset_dict.push_to_hub("eljanmahammadli/az-corpus-v2", private=True)

In [114]:
dataset

Dataset({
    features: ['text', 'source'],
    num_rows: 6783940
})

In [21]:
from nltk.tokenize import sent_tokenize, word_tokenize
from datasets import Dataset

def chunk_text(text, max_words=350):
    try:
        sentences = sent_tokenize(text)
        chunks = []
        current_chunk = []
        current_word_count = 0
        
        for sentence in sentences:
            # Use word_tokenize for a more accurate word count
            sentence_words = word_tokenize(sentence)
            sentence_word_count = len(sentence_words)
            
            if current_word_count + sentence_word_count > max_words:
                chunks.append(' '.join(current_chunk))  # Join the current chunk into one string
                current_chunk = [sentence]  # Start a new chunk with the current sentence
                current_word_count = sentence_word_count
            else:
                current_chunk.append(sentence)
                current_word_count += sentence_word_count
                
        # Don't forget to add the last chunk if it's not empty
        if current_chunk:
            chunks.append(' '.join(current_chunk))
        
        return chunks
    except:
        return [text]

def split_into_chunks(examples):
    all_chunks = []
    for text in examples['text']:
        text_chunks = chunk_text(text, 370)
        all_chunks.extend(text_chunks)
    return {'text': all_chunks}

chunked_dataset = dataset.map(split_into_chunks, batched=True, remove_columns=dataset.column_names)
chunked_dataset = chunked_dataset.flatten()

Map: 100%|██████████| 257606/257606 [15:33<00:00, 275.93 examples/s]


In [30]:
chunked_dataset

Dataset({
    features: ['text'],
    num_rows: 624822
})

In [ ]:
az_corpus_v2_chunk = load_dataset("eljanmahammadli/az-corpus-v2-chunk-512")['train']

In [29]:
az_corpus_v2_chunk

Dataset({
    features: ['text'],
    num_rows: 11703110
})

In [33]:
az_corpus_v2_and_wiki = concatenate_datasets([
    az_corpus_v2_chunk,
    chunked_dataset
])

In [49]:
az_corpus_v2_and_wiki

Dataset({
    features: ['text'],
    num_rows: 12327932
})

In [51]:
from datasets import DatasetDict

In [52]:
chunked_dataset_dict = DatasetDict({
    "train": az_corpus_v2_and_wiki
})
chunked_dataset_dict.push_to_hub("eljanmahammadli/az-corpus-v2-chunk-512", private=True)

































Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 77.43ba/s]































Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 81.64ba/s]































Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 80.41ba/s]






























Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 81.43ba/s]






























Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 83.14ba/s]






































Creating parquet from Arrow format: 100%|██████████| 268/268 [00:04<00:00, 61.98ba/s]






































Creating parquet from Arrow format: 100%|██████████| 268/268 [00:04<00:00, 62.53ba/s]






























Creating parquet from Arrow format: 100%|██████████| 268/268 [00:03<00:00, 85.95ba/s] 





















Creating parquet from Arrow fo

CommitInfo(commit_url='https://huggingface.co/datasets/eljanmahammadli/az-corpus-v2-chunk-512/commit/7773074ce8bedad5be72bb1a77c0529bdf154ce4', commit_message='Upload dataset', commit_description='', oid='7773074ce8bedad5be72bb1a77c0529bdf154ce4', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print("done")

## translated wiki

In [2]:
from datasets import load_dataset

dataset = load_dataset("BHOSAI/Translated_English_Wikipedia_on_Azerbaijani")['train']
dataset = dataset.remove_columns(["wikidata_id", "text", "version_id"])
dataset = dataset.rename_column("translation", "text")
dataset

/home/eljan/miniconda3/envs/azgpt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 257606 examples [01:51, 2318.64 examples/s]


In [17]:
# for cleaning the translated wikipedia text
import re

# Define the text cleaning and formatting function
def clean_and_format(example):
    # Extract the text from the example
    text = example["text"]
    
    # Apply the enhanced text cleaning and formatting
    text = re.sub(r'(_START_ARTICLE_|_START_SECTION_)(\w+)', r'\1\2:', text)
    text = re.sub(r"_[A-Z_]+_", " ", text)
    text = re.sub(r" +", " ", text).strip()
    
    # Update the example text
    example["text"] = text
    return example


# cleaned_dataset = dataset.select(range(100)).map(clean_and_format)
dataset = dataset.map(clean_and_format)


Map: 100%|██████████| 257606/257606 [01:02<00:00, 4121.06 examples/s]


In [20]:
dataset

Dataset({
    features: ['text'],
    num_rows: 257606
})